In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, clear_output

In [73]:
dataset= pd.read_csv("stroke prediction.csv")
dataset.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0


In [74]:
dataset.drop(columns=["id","Residence_type"],inplace=True)

In [75]:
dataset["gender"]=dataset["gender"].astype('category')
dataset["ever_married"]=dataset["ever_married"].astype('category')
# dataset["Residence_type"]=dataset["Residence_type"].astype('category')
dataset["work_type"]=dataset["work_type"].astype('category')
dataset["smoking_status"]=dataset.smoking_status.astype("category")

In [76]:
dataset_imputed=dataset.copy()

In [77]:
dataset_imputed.dtypes

gender               category
age                   float64
hypertension            int64
heart_disease           int64
ever_married         category
work_type            category
avg_glucose_level     float64
bmi                   float64
smoking_status       category
stroke                  int64
dtype: object

In [78]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()

dataset_imputed.gender=encoder.fit_transform(dataset_imputed.gender)

dataset_imputed.ever_married=encoder.fit_transform(dataset_imputed.ever_married)

dataset_imputed.work_type=encoder.fit_transform(dataset_imputed.work_type)

# dataset_imputed.Residence_type=encoder.fit_transform(dataset_imputed.Residence_type)

dataset_imputed.smoking_status=dataset_imputed.smoking_status.map({'never smoked':0,'formerly smoked':1,'smokes':2})

In [79]:
X=dataset_imputed.drop(columns="stroke",axis=1)
y=dataset_imputed["stroke"]

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=88,stratify=y)

In [80]:
from sklearn.impute import KNNImputer

imputer=KNNImputer()

X_trimputed=pd.DataFrame(imputer.fit_transform(X_train),columns=X_train.columns)

X_teimputed=pd.DataFrame(imputer.transform(X_test),columns=X_test.columns)

In [81]:
X_trimputed["smoking_status"]=X_trimputed["smoking_status"].apply(round)
X_trimputed["smoking_status"].value_counts()

X_trimputed["smoking_status"]=X_trimputed["smoking_status"].astype('category')
X_trimputed["work_type"]=X_trimputed["work_type"].apply(int)
X_trimputed["work_type"]=X_trimputed["work_type"].astype('category')

In [82]:
X_teimputed["smoking_status"]=X_teimputed["smoking_status"].apply(round)
X_teimputed["smoking_status"].value_counts()

X_teimputed["smoking_status"]=X_teimputed["smoking_status"].astype('category')
X_teimputed["work_type"]=X_teimputed["work_type"].apply(int)
X_teimputed["work_type"]=X_teimputed["work_type"].astype('category')

In [83]:
X_trimputed["gender"]=X_trimputed["gender"].astype("category")
X_trimputed[["hypertension","heart_disease","ever_married"]]=X_trimputed[["hypertension","heart_disease","ever_married"]].astype("int")

X_teimputed["gender"]=X_teimputed["gender"].astype("category")
X_teimputed[["hypertension","heart_disease","ever_married"]]=X_teimputed[["hypertension","heart_disease","ever_married"]].astype("int")

In [84]:
X_trimputed=pd.get_dummies(X_trimputed,drop_first=True)
X_teimputed=pd.get_dummies(X_teimputed,drop_first=True)

In [85]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()

scaler.fit(X_trimputed)

X_trimputed=pd.DataFrame(scaler.transform(X_trimputed),columns=X_trimputed.columns)

X_teimputed=pd.DataFrame(scaler.transform(X_teimputed),columns=X_teimputed.columns)

# Basic Ensembling

In [86]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.metrics import f1_score, classification_report, roc_auc_score

In [108]:
#model1
model1=LogisticRegression(class_weight='balanced',random_state=40)
model1.fit(X_trimputed,y_train)
pred1=model1.predict(X_teimputed)
f1_score(y_test,pred1)

model1_train=model1.predict(X_trimputed)

In [109]:
#model2
model2=KNeighborsClassifier(n_neighbors=3,algorithm='auto')
model2.fit(X_trimputed,y_train)
pred2=model2.predict(X_teimputed)
f1_score(y_test,pred2)

model2_train=model2.predict(X_trimputed)

In [110]:
#model3
model3=DecisionTreeClassifier(class_weight='balanced',ccp_alpha=0.10,random_state=40)
model3.fit(X_trimputed,y_train)
pred3=model3.predict(X_teimputed)
f1_score(y_test,pred3)

model3_train=model3.predict(X_trimputed)

In [111]:
#model4
model4=SVC(class_weight='balanced',random_state=40,kernel='poly',probability=True,degree=2)
model4.fit(X_trimputed,y_train)
pred4=model4.predict(X_teimputed)
f1_score(y_test,pred4)

model4_train=model4.predict(X_trimputed)

In [100]:
df = pd.DataFrame(
{'M1' : pred1,
'M2' : pred2,
'M3' : pred3,
'M4' : pred4, 
'Actual': y_test})

In [92]:
df

,M1,M2,M3,M4,Actual
28520,1,0,1,1,0
19948,0,0,0,0,0
10000,0,0,0,0,0
25636,0,0,0,0,0
29546,0,0,0,0,0
...,...,...,...,...,...
9878,0,0,0,0,0
6785,0,0,0,0,0
8563,0,0,0,0,0
24455,0,0,1,0,0


In [93]:
df[df['M4']==1]

,M1,M2,M3,M4,Actual
28520,1,0,1,1,0
30148,1,0,1,1,0
11787,1,0,1,1,0
19886,1,0,1,1,0
26285,1,0,1,1,0
...,...,...,...,...,...
37336,1,0,1,1,0
15484,1,0,1,1,0
32119,1,0,1,1,0
41507,1,0,1,1,0


In [102]:
df['M1_1']=df['M1'].copy()

In [103]:
df['M4_1']=df['M4'].copy()

In [104]:
df

,M1,M2,M3,M4,Actual,M1_1,M4_1
28520,1,0,1,1,0,1,1
19948,0,0,0,0,0,0,0
10000,0,0,0,0,0,0,0
25636,0,0,0,0,0,0,0
29546,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
9878,0,0,0,0,0,0,0
6785,0,0,0,0,0,0,0
8563,0,0,0,0,0,0,0
24455,0,0,1,0,0,0,0


In [105]:
df['Predicted']=round((df['M1']+df['M2']+df['M3']+df['M4']+df['M1_1']+df['M4_1'])/6,0).astype('int')
df

,M1,M2,M3,M4,Actual,M1_1,M4_1,Predicted
28520,1,0,1,1,0,1,1,1
19948,0,0,0,0,0,0,0,0
10000,0,0,0,0,0,0,0,0
25636,0,0,0,0,0,0,0,0
29546,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
9878,0,0,0,0,0,0,0,0
6785,0,0,0,0,0,0,0,0
8563,0,0,0,0,0,0,0,0
24455,0,0,1,0,0,0,0,0


In [98]:
f1_score(df['Actual'],df['Predicted'])

0.1094700260642919

# Basic Stacking

In [112]:
df_train = pd.DataFrame(
{'M1' : model1_train,
'M2' : model2_train,
'M3' : model3_train,
'M4' : model4_train, 
'Actual': y_train})

df_test = pd.DataFrame(
{'M1' : pred1,
'M2' : pred2,
'M3' : pred3,
'M4' : pred4, 
'Actual': y_test})

In [116]:
df_train

,M1,M2,M3,M4,Actual
8196,0,0,0,0,0
23697,0,0,0,0,0
28023,0,0,0,0,0
5054,1,0,1,1,0
33294,1,0,1,1,0
...,...,...,...,...,...
14160,0,0,1,0,0
21610,0,0,0,0,0
4408,0,0,0,0,0
4539,0,0,0,0,0


In [143]:
dt=LogisticRegression(class_weight='balanced',random_state=40)

dt.fit(df_train.drop(columns='Actual'),df_train['Actual'])
df_train['Predicted']=dt.predict(df_train.drop(columns='Actual'))
df_test['Predicted']=dt.predict(df_test.drop(columns='Actual'))

f1_score(df_train['Actual'],df_train['Predicted']),f1_score(df_test['Actual'],df_test['Predicted'])

(0.10097719869706841, 0.09962264150943395)

# Advanced Stacking

In [150]:
from sklearn.model_selection import KFold

def kfold_predictions( model , n_splits , train_x , train_y, test_x ):
    ## Initializing empty Numpy Arrays
    train_pred = np.empty((0,0) , int)
    skfold = KFold(n_splits)
  
    #For every permutation of KFold
    for i,j in skfold.split(train_x, train_y):
        x_train, x_test = train_x.iloc[i], train_x.iloc[j]
        y_train, y_test = train_y.iloc[i], train_y.iloc[j]
    
        #Train a model on training set
        model.fit( X = x_train, y = y_train)
    
        #Predict on Chosen Group
        tmp = model.predict(x_test)
    
        #Storing train predictions
        train_pred = np.append(train_pred, tmp)
    
    #Training on complete data and generating Predictionson Test set
    model.fit(train_x, train_y)
    test_pred = model.predict(test_x)
  
    return train_pred, test_pred

In [155]:
LR = LogisticRegression(class_weight='balanced')
M1_train, M1_test = kfold_predictions( LR, 5, X_trimputed, y_train, X_teimputed) 

In [156]:
DT = DecisionTreeClassifier(class_weight='balanced',ccp_alpha=0.10,random_state=40)
M2_train, M2_test = kfold_predictions( DT, 5, X_trimputed, y_train, X_teimputed) 

In [157]:
KNN = KNeighborsClassifier(n_neighbors=3)
M3_train, M3_test= kfold_predictions(KNN, 5, X_trimputed, y_train, X_teimputed) 

In [158]:
SVM = SVC(class_weight='balanced',random_state=40,kernel='poly',probability=True,degree=2)
M4_train, M4_test= kfold_predictions(SVM, 5, X_trimputed, y_train, X_teimputed)

In [172]:
# Arranging Train KFold Predictions
train_prediction = {
              'LR': M1_train,
              'KNN': M3_train,
              'DT': M2_train,
              'SVM':M4_train
              }
train_predictions = pd.DataFrame(train_prediction)
train_predictions.head()

,LR,KNN,DT,SVM
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,1,0,1,1
4,1,0,1,1


In [173]:
# Arranging Test KFold Predictions
test_prediction = {
              'LR': M1_test,
              'KNN': M3_test,
              'DT': M2_test,
              'SVM':M4_test
              }
test_predictions = pd.DataFrame(test_prediction)
test_predictions.head()

,LR,KNN,DT,SVM
0,1,0,1,1
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0


In [180]:
# Final Model taking the previous predictions as training set
model= DecisionTreeClassifier(class_weight='balanced',random_state=40)
model.fit(train_predictions, y_train)
final = model.predict(test_predictions)
f1_score(y_test,final)

0.09973554967888175